In [3]:
pip install pandas konlpy matplotlib wordcloud

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ------ --------------------------------- 3.1/19.4 MB 17.2 MB/s eta 0:00:01
   ----------- ---------------------------- 5.8/19.4 MB 14.0 MB/s eta 0:00:01
   ------------------ --------------------- 9.2/19.4 MB 14.5 MB/s eta 0:00:01
   ---------------------- ----------------- 11.0/19.4 MB 12.7 MB/s eta 0:00:01
   ----------------------------- ---------- 14.2/19.4 MB 13.0 MB/s eta 0:00:01
   ----------------------------------- ---- 17.0/19.4 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------  19.4/19.4 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 12.7 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [JPype1]
   ---------------------------------------- 0/3 [JPype1]
   ------------- -------------------------- 1/3 [konlpy]
   ------------- -------------------------- 1/3 [konlpy]
   ------------- -------------------------- 1/3 [konlpy]
   ------------

In [21]:
import re
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# 1. 텍스트 데이터
text_data =
"""

"""

# ==========================================
# 단계 1: 텍스트 구조화 (Parsing)
# ==========================================
def parse_record(text):
    lines = text.strip().split('\n')
    data = []
    current_grade = None
    current_category = None
    
    # 정규표현식 패턴
    grade_pattern = re.compile(r'\[(\d+학년)\]')
    category_pattern = re.compile(r'\[(.*활동)\]') # 자율활동, 동아리활동, 진로활동 등

    current_content = []

    for line in lines:
        line = line.strip()
        if not line: continue

        # 학년 감지
        grade_match = grade_pattern.match(line)
        if grade_match:
            current_grade = grade_match.group(1)
            continue

        # 활동 카테고리 감지
        cat_match = category_pattern.match(line)
        if cat_match:
            # 이전 내용 저장
            if current_category and current_content:
                data.append({
                    '학년': current_grade,
                    '활동': current_category,
                    '내용': " ".join(current_content)
                })
                current_content = []
            current_category = cat_match.group(1)
            continue

        # 내용 누적
        if current_grade and current_category:
            current_content.append(line)
            
    # 마지막 데이터 저장
    if current_category and current_content:
        data.append({
            '학년': current_grade,
            '활동': current_category,
            '내용': " ".join(current_content)
        })
        
    return pd.DataFrame(data)

df = parse_record(text_data)

# ==========================================
# 단계 2: 키워드(명사) 빈도 분석
# ==========================================
okt = Okt()

# 분석에서 제외할 불용어 (원하는 대로 추가 가능)
stopwords = ['대해', '통해', '위해', '가짐', '것', '등', '함', '수', '그', '자신', '이해', '수업', '대한', '활동', '노력', '주제', '생각', '진로', '과정', '내용', '관련', '능력', '문제', '학생', '직업', '활용', '분석', '설명', '모습', '학습', '방법', '탐구', '제작', '이용', '사용', '참여', '해결', '친구', '표현', '정리', '제시']

def extract_keywords(text):
    nouns = okt.nouns(text)
    return [n for n in nouns if len(n) > 1 and n not in stopwords]

# 전체 텍스트에서 키워드 추출
all_text = " ".join(df['내용'].tolist())
keywords = extract_keywords(all_text)
keyword_counts = Counter(keywords)

print("=== 상위 키워드 TOP 10 ===")
print(keyword_counts.most_common(10))

# ==========================================
# 단계 3: 역량 평가 어휘 분석 (감성 분석 대체)
# ==========================================
# 생기부에서 긍정적 평가(역량)를 나타내는 형용사/동사 리스트 정의(추가 필요)
competency_words = [
    '탁월한', '우수한', '주도적인', '성실한', '적극적', '심도있는', '논리적', 
    '명확하고', '향상', '이해', '분석', '창의적', '노력', '발전'
]

def analyze_competency(text):
    # 형태소 분석 (단어, 품사)
    pos = okt.pos(text, stem=True) # stem=True로 기본형 변환 (했다 -> 하다)
    
    found_competencies = []
    for word, tag in pos:
        # 형용사(Adjective)거나 동사(Verb)이면서, 역량 단어 리스트에 포함되거나
        # 혹은 긍정적인 서술어 패턴일 경우 추출
        if tag in ['Adjective', 'Verb']:
            # 간단히 역량 리스트와 매칭 (실제론 더 복잡한 로직 가능)
            for comp in competency_words:
                if comp.startswith(word) or word in comp: # '우수' -> '우수한' 매칭
                    found_competencies.append(word)
                    break
    return found_competencies

# 데이터프레임에 역량 키워드 컬럼 추가
df['발견된_역량'] = df['내용'].apply(analyze_competency)
df['역량_개수'] = df['발견된_역량'].apply(len)

print("\n=== 역량 평가 분석 결과 ===")
print(df[['학년', '활동', '역량_개수', '발견된_역량']])

# ==========================================
# (옵션) 시각화: 워드클라우드
# ==========================================
# 한글 폰트 경로 설정 필요 (Mac: AppleGothic, Win: Malgun Gothic)
# font_path = 'C:/Windows/Fonts/malgun.ttf' 
# wc = WordCloud(font_path=font_path, background_color='white', width=800, height=600)
# cloud = wc.generate_from_frequencies(keyword_counts)
# plt.figure(figsize=(10, 8))
# plt.imshow(cloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()

=== 상위 키워드 TOP 10 ===
[('게임', 53), ('인공', 31), ('지능', 31), ('데이터', 31), ('기술', 27), ('프로그래밍', 20), ('정보', 20), ('발표', 19), ('실험', 19), ('원리', 19)]

=== 역량 평가 분석 결과 ===
    학년     활동  역량_개수 발견된_역량
0  1학년   자율활동      0     []
1  1학년  동아리활동      0     []
2  2학년   진로활동      0     []
3  2학년   자율활동      0     []
4  2학년  동아리활동      0     []
5  3학년   진로활동      0     []
6  3학년   자율활동      0     []
7  3학년  동아리활동      0     []
8  3학년   진로활동      0     []
